In [1]:
import pandas as pd
import numpy as np

#### 读取数据

In [2]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv('data/users.dat', sep='::', header=None, names=unames)
users.head()

/Users/dengxiuqi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [3]:
users.shape

(6040, 5)

In [4]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('data/ratings.dat', sep='::', header=None, names=rnames, engine='python')
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
ratings.shape

(1000209, 4)

In [6]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('data/movies.dat', sep='::', header=None, names=mnames, engine='python')
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies.shape

(3883, 3)

#### 合并数据

In [8]:
df = pd.merge(pd.merge(ratings, users, on='user_id'), movies, on='movie_id')
df.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [9]:
df.shape

(1000209, 10)

#### 评分统计

In [10]:
df['rating'].value_counts()

4    348971
3    261197
5    226310
2    107557
1     56174
Name: rating, dtype: int64

#### 挑选有20～40次打分的movie_id

In [11]:
movie_group = df.groupby('movie_id').size()
movie_group = pd.Series(movie_group).where(lambda x : x<40).dropna()
movie_list=pd.Series(movie_group).where(lambda x : x>20).dropna().index.values
len(movie_list)

338

In [12]:
df = df[df['movie_id'].isin(movie_list)]
df.shape

(9974, 10)

#### 挑选有10次打分行为的user_id

In [13]:
user_group = df.groupby('user_id').size()
user_list = pd.Series(user_group).where(lambda x : x>10).dropna().index.values
len(user_list)

187

In [14]:
df = df[df['user_id'].isin(user_list)]
df.shape

(3944, 10)

#### 187个用户, 评价了338部电影

In [15]:
df['user_id'].nunique(), df['movie_id'].nunique()

(187, 338)

In [16]:
df.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
285581,1448,1509,5,976138368,F,25,3,17522,All Over Me (1997),Drama
285582,1758,1509,4,974853759,F,25,0,33067-1400,All Over Me (1997),Drama
285586,3519,1509,4,966957368,F,25,14,11215,All Over Me (1997),Drama
285588,3985,1509,3,965613710,M,45,16,01880,All Over Me (1997),Drama
285593,5317,1509,4,960907489,F,25,0,02138,All Over Me (1997),Drama


#### rating > 3定义为1(喜爱), 否则为-1(不喜爱)
* 为什么不是0？因为要让sigmoid函数更好的进行区分

In [17]:
df['rating']=df['rating'].map(lambda x: 1 if x>3 else -1)   #1,2, 3是label=1  4,5是label=0

#### 特征编码

In [20]:
from sklearn.preprocessing import OneHotEncoder
columns = ['user_id', 'movie_id']
for i in columns:
    get_dummy_feature = pd.get_dummies(df[i])
    df = pd.concat([df, get_dummy_feature],axis=1)
    df = df.drop(i, axis=1)

In [22]:
df.head()

,rating,timestamp,gender,age,occupation,zip,title,genres,45,117,...,3913,3922,3924,3931,3934,3935,3938,3939,3941,3942
285581,1,976138368,F,25,3,17522,All Over Me (1997),Drama,0,0,...,0,0,0,0,0,0,0,0,0,0
285582,1,974853759,F,25,0,33067-1400,All Over Me (1997),Drama,0,0,...,0,0,0,0,0,0,0,0,0,0
285586,1,966957368,F,25,14,11215,All Over Me (1997),Drama,0,0,...,0,0,0,0,0,0,0,0,0,0
285588,-1,965613710,M,45,16,01880,All Over Me (1997),Drama,0,0,...,0,0,0,0,0,0,0,0,0,0
285593,1,960907489,F,25,0,02138,All Over Me (1997),Drama,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df.shape

(3944, 533)

#### 暂不考虑除评分外的其他特征

In [24]:
df=df.drop(['timestamp','gender','age','occupation','zip','title','genres'], axis=1) 

#### 测试集划分

In [25]:
from sklearn.model_selection import train_test_split

X=df.drop('rating', axis=1)
Y=df['rating']

X_train,X_val,Y_train,Y_val=train_test_split(X, Y, test_size=0.1, random_state=123)
X_train.shape, X_val.shape

((3549, 525), (395, 525))

In [26]:
Y.value_counts()

-1    2471
 1    1473
Name: rating, dtype: int64

#### FM

In [27]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def logit(y, y_hat): #对每一个样本计算损失
    return np.log(1 + np.exp(-y * y_hat))

def df_logit(y, y_hat):
    return sigmoid(-y * y_hat) * (-y)

In [30]:
from sklearn.base import BaseEstimator,ClassifierMixin
from collections import Counter


class FactorizationMachine(BaseEstimator):
    def __init__(self, k=5, learning_rate=0.01, iternum=100): 
        self.w0 = None 
        self.W = None 
        self.V = None 
        self.k = k 
        self.alpha = learning_rate
        self.iternum = iternum
        
        
    def _FM(self, Xi):
        interaction = 0.5 * np.sum((Xi.dot(self.V)) ** 2 - (Xi ** 2).dot(self.V ** 2))  # 交叉项
        y_hat = self.w0 + Xi.dot(self.W) + interaction
        return y_hat[0]
        
        
    def _FM_SGD(self, X, y):
        m, n = np.shape(X) 
        #初始化参数
        self.w0 = 0
        self.W = np.random.uniform(size=(n, 1))
        self.V = np.random.uniform(size=(n, self.k)) #Vj是第j个特征的隐向量  Vjf是第j个特征的隐向量表示中的第f维
        
        for it in range(self.iternum):
            total_loss = 0  
            for i in range(m):  # 遍历训练集
                y_hat = self._FM(Xi=X[i])  #X[i]是第i个样本  X[i,j]是第i个样本的第j个特征

                total_loss += logit(y=y[i], y_hat=y_hat)  # 计算logit损失函数值
                dloss = df_logit(y=y[i], y_hat=y_hat)  # 计算logit损失函数的外层偏导

                dloss_w0 = dloss * 1   # 公式中的w0求导，计算复杂度O(1)
                self.w0 = self.w0 - self.alpha * dloss_w0 

                for j in range(n): 
                    if X[i, j] != 0:
                        dloss_Wj = dloss * X[i, j]  # 公式中的wi求导，计算复杂度O(n)
                        self.W[j] = self.W[j] - self.alpha * dloss_Wj 
                        for f in range(self.k):  # 公式中的vif求导，计算复杂度O(kn)
                            dloss_Vjf = dloss * (X[i, j] * (X[i].dot(self.V[:, f])) - self.V[j, f] * X[i, j] ** 2)
                            self.V[j, f] = self.V[j, f] - self.alpha * dloss_Vjf 

            if it % 10 == 0:
                print('iter={}, loss={:.4f}'.format(it, total_loss / m))

        return self
    
    
    def _FM_predict(self, X):
        predicts, threshold = [], 0.5  # sigmoid阈值设置
        for i in range(X.shape[0]):  # 遍历测试集
            y_hat = self._FM(Xi=X[i])  # FM的模型方程 
            predicts.append(-1 if sigmoid(y_hat) < threshold else 1) 
        return np.array(predicts)        
        
        
    def fit(self, X, y):
        if isinstance(X, pd.DataFrame):
            X = np.array(X)
            y = np.array(y)
            
        return self._FM_SGD(X, y)

    
    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = np.array(X)
        
        return self._FM_predict(X)
    
    
    def predict_proba(self, X):
        pass

#### Train & Predict

In [31]:
model=FactorizationMachine(k=10, learning_rate=0.01, iternum=150)
model.fit(X_train, Y_train)

iter=0, loss=0.8226
iter=10, loss=0.4659
iter=20, loss=0.4096
iter=30, loss=0.3703
iter=40, loss=0.3330
iter=50, loss=0.2954
iter=60, loss=0.2583
iter=70, loss=0.2230
iter=80, loss=0.1908
iter=90, loss=0.1624
iter=100, loss=0.1381
iter=110, loss=0.1176
iter=120, loss=0.1007
iter=130, loss=0.0867
iter=140, loss=0.0752


FactorizationMachine(iternum=150, k=10, learning_rate=None)

In [32]:
from sklearn.metrics import classification_report
y_pred=model.predict(X_train)
report  = classification_report(Y_train, y_pred)
print(report)

             precision    recall  f1-score   support

         -1       1.00      1.00      1.00      2213
          1       1.00      1.00      1.00      1336

avg / total       1.00      1.00      1.00      3549



In [34]:
y_pred=model.predict(X_val)
report  = classification_report(Y_val, y_pred)
print(report)

             precision    recall  f1-score   support

         -1       0.76      0.79      0.78       258
          1       0.58      0.54      0.56       137

avg / total       0.70      0.71      0.70       395



#### 以某个用户为例

In [47]:
X_val.iloc[0:10]

,45,117,173,187,195,225,278,293,327,352,...,3913,3922,3924,3931,3934,3935,3938,3939,3941,3942
948069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
974730,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997632,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
948658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
956196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
963451,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996177,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
987363,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
958439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
y_pred=model.predict(X_val.iloc[0:10])

In [51]:
y_pred

array([-1,  1, -1, -1,  1, -1, -1, -1,  1, -1])

In [54]:
model.V.shape

(525, 10)